<a href="https://colab.research.google.com/github/KonstantinBurkin/Machine_Learning_Project/blob/main/Machine_Learning_Delivery_Club.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Sales Forecasting

In this project I build a Machine Learning method that predicts the number of orders for each store in Delivery Club and each product for the next week. Here, I use data set describing the number of sales of goods in stores on the Delivery Club platform in 10 cities in Russia (Moscow, St. Petersburg, Krasnodar, Samara, Nizhny Novgorod, Rostov-on-Don, Volgograd, Voronezh, Kazan, Yekaterinburg).

Two data sets (train.csv and test.csv) are used to train Regression model and evaluate its accuracy. Both data sets have the same list of variables:
- id - уникальный идентификатор, представляющий связку (product_id,
store_id, date). Это значит, что для каждой тройки (product_id, store_id, date) существует лишь один id, он не повторяется в данных
- date - дата продажи продукта
- city_name - название города, в котором происходила продажа
- store_id - уникальный идентификатор для каждого магазина
- category_id - категория продаваемого товара
- product_id - уникальный идентификатор товара
- price - цена товара
- weather_desc - краткое описание погоды в этом городе в день продажи
- humidity - влажность в этом городе в день продажи
- temperature - температура в этом городе в день продажи
- pressure - атмосферное давление в этом городе в день продажи
- sales - количество продаж товара (это то, что нужно прогнозировать)

- MAE - средняя абсолютная ошибка, показывает на сколько заказов в среднем ошибается прогноз. Это метрика очень легко интерпретируема. Если у нас значение метрики равно, например, 5, то это значит модель в среднем ошибается на 5 заказов каждый час по каждому району. Метрика не бывает отрицательной, так как все ошибки беруться по модулю, для идеальной модели эта метрика будет равна 0. Метрика не так чувствительна к выбросам.

$$MAE = \frac1N \sum ^{N}_{i=1} |y_i-\hat y_i|$$

In [1]:
# Import libraries

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
import plotly.express as px
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from google.colab import output
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
from google.colab import files
uploaded = files.upload()

In [ ]:
# Download data from Github

# !wget --no-cache --backups=1 {"https://github.com/KonstantinBurkin/Machine_Learning_Project/blob/main/train.csv"}
# !wget --no-cache --backups=1 {"https://github.com/KonstantinBurkin/Machine_Learning_Project/blob/main/public_data.zip"}
# output.clear()

In [2]:
# Upload data sets into the project

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

## Data sets description

In [4]:
train = train.replace(
    ('Москва', 'Санкт-Петербург', 'Краснодар', 'Самара','Нижний Новгород', 'Ростов-на-Дону', 'Волгоград', 'Воронеж', 'Казань', 'Екатеринбург'),
    ("Moscow", "St.Petersburg", "Krasnodar", "Samara", "Nizhny.Novgorod", "Rostov-on-Don", "Volgograd", "Voronezh", "Kazan", "Yekaterinburg")  )

In [5]:
pd.unique(train["city_name"])
# pd.unique(train["store_id"])
# можно сделать график количества магазинов в каждом городе

array(['Moscow', 'St.Petersburg', 'Krasnodar', 'Samara',
       'Nizhny.Novgorod', 'Rostov-on-Don', 'Volgograd', 'Voronezh',
       'Kazan', 'Yekaterinburg'], dtype=object)

In [3]:
train.shape

(666676, 12)

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28489 entries, 0 to 28488
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            28489 non-null  int64  
 1   date          28488 non-null  object 
 2   city_name     28488 non-null  object 
 3   store_id      28488 non-null  float64
 4   category_id   28488 non-null  float64
 5   product_id    28488 non-null  float64
 6   price         28488 non-null  float64
 7   weather_desc  28488 non-null  object 
 8   humidity      28488 non-null  float64
 9   temperature   28488 non-null  float64
 10  pressure      28488 non-null  float64
 11  sales         28488 non-null  float64
dtypes: float64(8), int64(1), object(3)
memory usage: 2.6+ MB


In [ ]:
train.head()

,id,date,city_name,store_id,category_id,product_id,price,weather_desc,humidity,temperature,pressure,sales
0,1,2021-07-29,Moscow,1.0,1.0,1.0,4.79,"переменная облачность, небольшой дождь",61.9375,23.1875,741.0000,26.0
1,2,2021-07-30,Moscow,1.0,1.0,1.0,4.79,"переменная облачность, небольшой дождь",70.2500,22.1875,740.3125,37.0
2,3,2021-07-31,Moscow,1.0,1.0,1.0,4.79,переменная облачность,52.6250,21.8125,741.6250,25.0
3,4,2021-08-01,Moscow,1.0,1.0,1.0,4.79,"облачно, небольшой дождь",87.4375,20.0625,743.3125,26.0
4,5,2021-08-02,Moscow,1.0,1.0,1.0,4.79,переменная облачность,66.1875,23.4375,739.6250,22.0


In [ ]:
train.describe()

,id,store_id,category_id,product_id,price,humidity,temperature,pressure,sales
count,28489.000000,28488.000000,28488.000000,28488.000000,28488.000000,28488.000000,28488.000000,28488.000000,28488.000000
mean,14244.000070,3.615768,2.398238,17.317397,5.104391,73.685486,7.737734,754.513429,12.057217
std,8224.210124,1.955576,1.902261,10.883911,3.345185,16.582210,10.770423,9.361326,15.567595
min,1.000000,1.000000,1.000000,1.000000,1.930000,27.125000,-18.312500,718.062500,0.000000
25%,7122.000000,2.000000,1.000000,9.000000,3.000000,61.562500,0.375000,748.312500,3.000000
50%,14244.000000,3.000000,1.000000,16.000000,4.090000,73.062500,7.687500,756.000000,7.000000
75%,21366.000000,5.000000,4.000000,28.000000,6.200000,89.312500,15.062500,761.000000,15.000000
max,28488.000000,7.000000,8.000000,35.000000,18.630000,100.000000,33.250000,779.000000,169.000000


In [ ]:
test.shape

(24836, 11)

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24836 entries, 0 to 24835
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            24836 non-null  int64  
 1   date          24836 non-null  object 
 2   city_name     24836 non-null  object 
 3   store_id      24836 non-null  int64  
 4   category_id   24836 non-null  int64  
 5   product_id    24836 non-null  int64  
 6   price         24836 non-null  float64
 7   weather_desc  24836 non-null  object 
 8   humidity      24836 non-null  float64
 9   temperature   24836 non-null  float64
 10  pressure      24836 non-null  float64
dtypes: float64(4), int64(4), object(3)
memory usage: 2.1+ MB


In [ ]:
test.head()

,id,date,city_name,store_id,category_id,product_id,price,weather_desc,humidity,temperature,pressure
0,666677,2022-02-14,Москва,1,1,1,4.79,облачно,87.3125,-1.9375,749.3125
1,666678,2022-02-15,Москва,1,1,1,4.79,переменная облачность,88.7500,-1.2500,752.6875
2,666679,2022-02-16,Москва,1,1,1,4.79,переменная облачность,90.3750,-1.5625,746.3125
3,666680,2022-02-17,Москва,1,1,1,4.79,"облачно, небольшой дождь",98.0000,1.7500,732.6875
4,666681,2022-02-18,Москва,1,1,1,4.79,"облачно, небольшие осадки",95.5000,1.3750,733.0000


In [ ]:
test.describe()

,id,store_id,category_id,product_id,price,humidity,temperature,pressure
count,24836.000000,24836.000000,24836.000000,24836.000000,24836.000000,24836.000000,24836.000000,24836.000000
mean,679094.500000,78.053551,2.375423,17.832300,5.201144,87.285168,-0.751719,747.908286
std,7169.679979,45.689019,1.876578,10.826993,3.491933,9.839292,4.059063,9.743387
min,666677.000000,1.000000,1.000000,1.000000,1.930000,55.875000,-10.500000,730.312500
25%,672885.750000,40.000000,1.000000,9.000000,3.000000,84.812500,-3.625000,740.000000
50%,679094.500000,76.000000,1.000000,17.000000,4.090000,89.937500,-0.312500,748.937500
75%,685303.250000,117.000000,4.000000,28.000000,6.020000,94.312500,1.750000,754.687500
max,691512.000000,164.000000,9.000000,35.000000,18.630000,98.625000,9.062500,769.000000


In [ ]:
group.head()

NameError: ignored

In [137]:
# график заказов по городам
group = train[['date', 'city_name', 'sales']].groupby(['date', 'city_name'], as_index=False).sum()
fig = px.line(group, x="date", y="sales", color='city_name', template='plotly_dark')
fig.update_layout(
    title={
        'text': "Orders in each city",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show() 
# можно еще добавить инфу как менялись заказы по месяцам, в течении недели

## Building regression model

In [4]:
# make 4 subsets for training and testing
X_train, X_test, y_train, y_test = train_test_split(
    train.iloc[:,0:11],
    train.iloc[:,11],
    train_size = 0.8, 
    test_size = 0.2,
    random_state = 2022,
    shuffle = True)


In [5]:
# Linear model

model = LinearRegression()
model.fit(X_train.iloc[:,[0,3,4,5,6,8,9,10]].dropna(axis=0, how='any', subset=None, inplace=False), # выкидываю character data type and NA
          y_train.dropna(axis=0, how='any', inplace=False)) # выкидываю character data type and NA

forecast_lm = model.predict(X_test.iloc[:,[0,3,4,5,6,8,9,10]].dropna(axis=0, how='any', subset=None, inplace=False))
mae = mean_absolute_error(y_test, forecast_lm)

print(f"Linear model: MAE = {mae:.2f} < 4.10 " if mae < 4.10 else f"Linear model: MAE = {mae:.2f} > 4.10 ")


Linear model: MAE = 8.70 > 4.10 


In [6]:
# TreeClassifier model
tree_clf = DecisionTreeClassifier().fit(X_train.iloc[:,[0,3,4,5,6,8,9,10]].dropna(axis=0, how='any', subset=None, inplace=False), 
                                        y_train.dropna(axis=0, how='any', inplace=False))
forecast_tree = tree_clf.predict(X_test.iloc[:,[0,3,4,5,6,8,9,10]].dropna(axis=0, how='any', subset=None, inplace=False))
mae = mean_absolute_error(y_test, forecast_tree)
print(f"TreeClassifier model: MAE = {mae:.2f} < 4.10 " if mae < 4.10 else f"TreeClassifier model: MAE = {mae:.2f} > 4.10 ")

TreeClassifier model: MAE = 5.24 > 4.10 


In [8]:
# KNN model
knn_clf = KNeighborsClassifier().fit(X_train.iloc[:,[0,3,4,5,6,8,9,10]].dropna(axis=0, how='any', subset=None, inplace=False), 
                                        y_train.dropna(axis=0, how='any', inplace=False))
forecast_knn = knn_clf.predict(X_test.iloc[:,[0,3,4,5,6,8,9,10]].dropna(axis=0, how='any', subset=None, inplace=False))
mae = mean_absolute_error(y_test, forecast_knn)
print(f"KNN model: MAE = {mae:.2f} < 4.10 " if mae < 4.10 else f"KNN model: MAE = {mae:.2f} > 4.10 ")

KNN model: MAE = 5.18 > 4.10 


In [9]:
# Sum of KNN and TreeClassifier
sum_of_voices = (forecast_knn + forecast_tree)/2
mae = mean_absolute_error(y_test, sum_of_voices)
print(f"KNN model: MAE = {mae:.2f} < 4.10 " if mae < 4.10 else f"KNN model: MAE = {mae:.2f} > 4.10 ")

KNN model: MAE = 4.49 > 4.10 


In [10]:
# RandomForestClassifier model
rf_model = RandomForestClassifier(n_estimators = 4)
rf_model.fit(X_train.iloc[:,[0,3,4,5,6,8,9,10]].dropna(axis=0, how='any', subset=None, inplace=False), 
                                        y_train.dropna(axis=0, how='any', inplace=False))
forecast = rf_model.predict(X_test.iloc[:,[0,3,4,5,6,8,9,10]].dropna(axis=0, how='any', subset=None, inplace=False))
mae = mean_absolute_error(y_test, forecast)
print(f"KNN model: MAE = {mae:.2f} < 4.10 " if mae < 4.10 else f"KNN model: MAE = {mae:.2f} > 4.10 ")

KNN model: MAE = 5.81 > 4.10 


## Results

In [ ]:
# df.to_csv("prediction.csv")